In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [9]:
data=pd.read_csv('./properatti.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [10]:
capital_federal_mask = data.state_name=="Capital Federal"
capital_federal = data[capital_federal_mask]
capital_federal

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121160,121160,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj5v_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,2611.464968,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...


In [11]:
capital_federal_dos = capital_federal.loc[:,('property_type','place_name','surface_total_in_m2','surface_covered_in_m2')]
capital_federal_dos

,property_type,place_name,surface_total_in_m2,surface_covered_in_m2
0,PH,Mataderos,55.0,40.0
2,apartment,Mataderos,55.0,55.0
3,PH,Liniers,NaN,NaN
7,apartment,Belgrano,45.0,40.0
8,apartment,Belgrano,65.0,60.0
...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN
121209,apartment,Belgrano,157.0,NaN
121215,apartment,Belgrano,113.0,93.0
121217,apartment,Villa Urquiza,46.0,39.0


In [12]:
capital_federal_apartment_mask = capital_federal.property_type=='apartment'
capital_federal_apartment = capital_federal_dos[capital_federal_apartment_mask]
capital_federal_apartment


,property_type,place_name,surface_total_in_m2,surface_covered_in_m2
2,apartment,Mataderos,55.0,55.0
7,apartment,Belgrano,45.0,40.0
8,apartment,Belgrano,65.0,60.0
13,apartment,Palermo Soho,50.0,30.0
14,apartment,Palermo Soho,42.0,31.0
...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN
121209,apartment,Belgrano,157.0,NaN
121215,apartment,Belgrano,113.0,93.0
121217,apartment,Villa Urquiza,46.0,39.0


In [13]:
dummies = pd.get_dummies(capital_federal_apartment['place_name']).rename(columns=lambda x: 'Category_' + str(x))
dummies
#df = pd.concat([df, dummies], axis=1)
#df = df.drop(['Category'], inplace=True, axis=1)

,Category_Abasto,Category_Agronomía,Category_Almagro,Category_Balvanera,Category_Barracas,Category_Barrio Norte,Category_Belgrano,Category_Boca,Category_Boedo,Category_Caballito,...,Category_Villa Lugano,Category_Villa Luro,Category_Villa Ortuzar,Category_Villa Pueyrredón,Category_Villa Real,Category_Villa Riachuelo,Category_Villa Santa Rita,Category_Villa Soldati,Category_Villa Urquiza,Category_Villa del Parque
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121160,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121209,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121215,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
#Porcentaje de nulls respecto al total del dataset por columna

percent_missing = capital_federal_apartment.isnull().sum() * 100 / len(capital_federal_apartment)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
surface_total_in_m2,surface_total_in_m2,16.484817
surface_covered_in_m2,surface_covered_in_m2,6.646447
property_type,property_type,0.000000
place_name,place_name,0.000000


In [15]:
capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2
capital_federal_apartment

C:\Users\Juan\AppData\Local\Temp\ipykernel_12776\3450111080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2


,property_type,place_name,surface_total_in_m2,surface_covered_in_m2,surface_uncovered_in_m2
2,apartment,Mataderos,55.0,55.0,0.0
7,apartment,Belgrano,45.0,40.0,5.0
8,apartment,Belgrano,65.0,60.0,5.0
13,apartment,Palermo Soho,50.0,30.0,20.0
14,apartment,Palermo Soho,42.0,31.0,11.0
...,...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN,NaN
121209,apartment,Belgrano,157.0,NaN,NaN
121215,apartment,Belgrano,113.0,93.0,20.0
121217,apartment,Villa Urquiza,46.0,39.0,7.0


In [16]:
percent_missing = capital_federal_apartment.isnull().sum() * 100 / len(capital_federal_apartment)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
surface_uncovered_in_m2,surface_uncovered_in_m2,21.174687
surface_total_in_m2,surface_total_in_m2,16.484817
surface_covered_in_m2,surface_covered_in_m2,6.646447
property_type,property_type,0.000000
place_name,place_name,0.000000


In [17]:
capital_federal_apartment.surface_total_in_m2.fillna(capital_federal_apartment.surface_covered_in_m2, inplace=True)


C:\Users\Juan\AppData\Local\Temp\ipykernel_12776\3128688942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment.surface_total_in_m2.fillna(capital_federal_apartment.surface_covered_in_m2, inplace=True)


In [18]:
capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2
capital_federal_apartment

C:\Users\Juan\AppData\Local\Temp\ipykernel_12776\3450111080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment['surface_uncovered_in_m2'] = capital_federal_apartment.surface_total_in_m2 - capital_federal_apartment.surface_covered_in_m2


,property_type,place_name,surface_total_in_m2,surface_covered_in_m2,surface_uncovered_in_m2
2,apartment,Mataderos,55.0,55.0,0.0
7,apartment,Belgrano,45.0,40.0,5.0
8,apartment,Belgrano,65.0,60.0,5.0
13,apartment,Palermo Soho,50.0,30.0,20.0
14,apartment,Palermo Soho,42.0,31.0,11.0
...,...,...,...,...,...
121160,apartment,Belgrano,157.0,NaN,NaN
121209,apartment,Belgrano,157.0,NaN,NaN
121215,apartment,Belgrano,113.0,93.0,20.0
121217,apartment,Villa Urquiza,46.0,39.0,7.0


In [19]:
capital_federal_apartment.drop('surface_total_in_m2', axis=1, inplace=True)

C:\Users\Juan\AppData\Local\Temp\ipykernel_12776\86561127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment.drop('surface_total_in_m2', axis=1, inplace=True)


In [20]:
percent_missing = capital_federal_apartment.isnull().sum() * 100 / len(capital_federal_apartment)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
surface_covered_in_m2,surface_covered_in_m2,6.646447
surface_uncovered_in_m2,surface_uncovered_in_m2,6.646447
property_type,property_type,0.000000
place_name,place_name,0.000000


In [21]:
capital_federal_apartment_clean = capital_federal_apartment.dropna()
capital_federal_apartment_clean

,property_type,place_name,surface_covered_in_m2,surface_uncovered_in_m2
2,apartment,Mataderos,55.0,0.0
7,apartment,Belgrano,40.0,5.0
8,apartment,Belgrano,60.0,5.0
13,apartment,Palermo Soho,30.0,20.0
14,apartment,Palermo Soho,31.0,11.0
...,...,...,...,...
121157,apartment,Belgrano,35.0,3.0
121158,apartment,Recoleta,39.0,5.0
121215,apartment,Belgrano,93.0,20.0
121217,apartment,Villa Urquiza,39.0,7.0


In [22]:
percent_missing = capital_federal_apartment_clean.isnull().sum() * 100 / len(capital_federal_apartment_clean)
missing_value_df = pd.DataFrame({'column_name': capital_federal_apartment_clean.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
property_type,property_type,0.0
place_name,place_name,0.0
surface_covered_in_m2,surface_covered_in_m2,0.0
surface_uncovered_in_m2,surface_uncovered_in_m2,0.0


In [23]:
dummies = pd.get_dummies(capital_federal_apartment_clean['place_name']).rename(columns=lambda x: 'Barrio_' + str(x))
dummies

#df = df.drop(['Category'], inplace=True, axis=1)

,Barrio_Abasto,Barrio_Agronomía,Barrio_Almagro,Barrio_Balvanera,Barrio_Barracas,Barrio_Barrio Norte,Barrio_Belgrano,Barrio_Boca,Barrio_Boedo,Barrio_Caballito,...,Barrio_Villa Lugano,Barrio_Villa Luro,Barrio_Villa Ortuzar,Barrio_Villa Pueyrredón,Barrio_Villa Real,Barrio_Villa Riachuelo,Barrio_Villa Santa Rita,Barrio_Villa Soldati,Barrio_Villa Urquiza,Barrio_Villa del Parque
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121157,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121215,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
capital_federal_apartment_clean.drop('property_type', inplace=True, axis=1)
capital_federal_apartment_clean

C:\Users\Juan\AppData\Local\Temp\ipykernel_12776\3058871001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capital_federal_apartment_clean.drop('property_type', inplace=True, axis=1)


,place_name,surface_covered_in_m2,surface_uncovered_in_m2
2,Mataderos,55.0,0.0
7,Belgrano,40.0,5.0
8,Belgrano,60.0,5.0
13,Palermo Soho,30.0,20.0
14,Palermo Soho,31.0,11.0
...,...,...,...
121157,Belgrano,35.0,3.0
121158,Recoleta,39.0,5.0
121215,Belgrano,93.0,20.0
121217,Villa Urquiza,39.0,7.0


In [25]:
numericals = ['surface_uncovered_in_m2', 'surface_covered_in_m2']

X = capital_federal_apartment_clean[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]

std_df


,surface_uncovered_in_m2_std,surface_covered_in_m2_std
0,0.819590,0.000294
1,0.820729,0.000214
2,0.820729,0.000321
3,0.824146,0.000160
4,0.822096,0.000166
...,...,...
25235,0.820273,0.000187
25236,0.820729,0.000209
25237,0.824146,0.000497
25238,0.821185,0.000209


X = pd.concat([dummies, std_df], axis = 1)
X

numericals = ['carat', 'depth', 'table', 'x', 'y', 'z']

X = data[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]
std_df

capital_federal_apartment_clean